In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.fftpack import dct, idct
import bitstring
import os
import io
from PIL import Image
import heapq
from collections import defaultdict

### JPEG Class

In [ ]:
class JPEG:
    def __init__(self, Q = 50):
        self.Q = Q
        self.quantization_matrix = np.array([   [16, 11, 10, 16, 24, 40, 51, 61],
                                                [12, 12, 14, 19, 26, 58, 60, 55],
                                                [14, 13, 16, 24, 40, 57, 69, 56],
                                                [14, 17, 22, 29, 51, 87, 80, 62],
                                                [18, 22, 37, 56, 68, 109, 103, 77],
                                                [24, 35, 55, 64, 81, 104, 113, 92],
                                                [49, 64, 78, 87, 103, 121, 120, 101],
                                                [72, 92, 95, 98, 112, 100, 103, 99],])
        self.scaled_quantization_matrix = np.round(self.quantization_matrix * (50.0/Q))
    
    # define the dct and idct functions for the blocks
    def _dct2(self, block):
        dct_val = dct(dct(block.T, norm='ortho').T, norm='ortho')
        return dct_val

    def _idct2(self, block):
        return idct(idct(block.T, norm='ortho').T, norm='ortho')
    
    # Runs quantization while compression
    def _quantize(self, block):
        return np.round(block / self.scaled_quantization_matrix)

    # Runs dequantization while decompression
    def _dequantize(self, block):
        return block * self.scaled_quantization_matrix
    
    # breaks the image into 8x8 patches
    def _chunkify(self, image):
        chunks = []
        h,w = np.shape(image)
        for i in range(0, h, 8):
            for j in range(0, w, 8):
                chunk = image[i:i+8, j:j+8]
                chunks.append(chunk)

        return chunks

    # recombines the patches into the original image size
    def _dechunkify(self, chunks, original_shape):
        h, w = original_shape
        image = np.zeros((h, w), dtype=chunks[0].dtype)  # Create an empty array with the original shape
        chunk_idx = 0  # Index to keep track of the current chunk

        for i in range(0, h, 8):
            for j in range(0, w, 8):
                # Place the current chunk into the corresponding position in the image
                image[i:i+8, j:j+8] = chunks[chunk_idx]
                chunk_idx += 1

        return image

    # flattens the matrix into its zig-zag traversal
    def _zigzag(self, matrix):
        rows, cols = matrix.shape
        result = []

        for s in range(rows + cols - 1):
            if s % 2 == 0:  # Even diagonals (move up-right)
                x = min(s, rows - 1)
                y = s - x
                while x >= 0 and y < cols:
                    result.append(matrix[x, y])
                    x -= 1
                    y += 1
            else:  # Odd diagonals (move down-left)
                y = min(s, cols - 1)
                x = s - y
                while y >= 0 and x < rows:
                    result.append(matrix[x, y])
                    x += 1
                    y -= 1

        return np.array(result)

    # converts a flattened matrix into the original form by placing the elements in a zig zag manner
    def _reverse_zigzag(self, flattened, rows, cols):
        # Initialize an empty 2D matrix
        matrix = np.zeros((rows, cols), dtype=flattened.dtype)

        # Fill the matrix using the reverse zig-zag order
        index = 0
        for s in range(rows + cols - 1):
            if s % 2 == 0:  # Even diagonals (move up-right)
                x = min(s, rows - 1)
                y = s - x
                while x >= 0 and y < cols:
                    matrix[x, y] = flattened[index]
                    index += 1
                    x -= 1
                    y += 1
            else:  # Odd diagonals (move down-left)
                y = min(s, cols - 1)
                x = s - y
                while y >= 0 and x < rows:
                    matrix[x, y] = flattened[index]
                    index += 1
                    x += 1
                    y -= 1

        return matrix
    
    # Helper functions for huffman encoding and RLE
    def _build_huffman_tree(self, frequencies):
        # Create a priority queue (min-heap) to build the Huffman tree
        heap = [[weight, [symbol, ""]] for symbol, weight in frequencies.items()]
        heapq.heapify(heap)
        
        while len(heap) > 1:
            low = heapq.heappop(heap)
            high = heapq.heappop(heap)
            for pair in low[1:]:
                pair[1] = '0' + pair[1]
            for pair in high[1:]:
                pair[1] = '1' + pair[1]
            heapq.heappush(heap, [low[0] + high[0]] + low[1:] + high[1:])
        
        # Generate the Huffman codes
        huffman_codes = {}
        for symbol, code in heap[0][1:]:
            huffman_codes[symbol] = code
        return huffman_codes

    def _run_length_encoding(self, matrix):
        rle = []
        zero_count = 0
        
        for value in matrix:
            if value == 0:
                zero_count += 1  # Increment zero counter
                if zero_count == 16:
                    rle.append((15, 0))
                    zero_count = 0
            else:
                rle.append((zero_count, value))  # Store the number of zeros before the value
                zero_count = 0  # Reset zero counter after encountering a non-zero value
        
        return rle

    def _huffman_encoding(self, values):
        frequencies = defaultdict(int)
        for val in values:
            frequencies[val] += 1
        
        huffman_codes = self._build_huffman_tree(frequencies)
        return huffman_codes

    # Function to encode the image post scaling dct and quantization into rle and huffman and store into a bin file
    def _encode(self, flattened_matrices, filename):
        with open(filename, 'wb') as f:
            rle_result = []

            for i in range(0,len(flattened_matrices)):
                rle = self._run_length_encoding(flattened_matrices[i])
                rle_result.extend(rle)
                rle_result.append((0,0))

            values = [value for count, value in rle_result]  # Only values for Huffman encoding
            huffman_codes = self._huffman_encoding(values)    

            # Store the Quality Factor in the file 
            f.write(np.array([self.Q], dtype=np.uint8).tobytes()) 
            f.flush()
        

            # Store the Huffman table in the file (value -> huffman code mapping)
            f.write(np.array([len(huffman_codes)], dtype=np.uint16).tobytes())  # Write the number of unique values in Huffman table
            f.flush()            
 

            # Collect all Huffman codes
            all_huffman_codes = []

            for value, code in huffman_codes.items():
                f.write(np.array([value], dtype=np.int16).tobytes())  
                f.flush()
                size_in_bits = len(code)
                f.write(np.array([size_in_bits], dtype=np.uint8).tobytes())  # Write the size of the Huffman code
                f.flush()
                all_huffman_codes.append(code)

            all_huffman_codes_str = ''.join(all_huffman_codes)
            total_size_in_bits = len(all_huffman_codes_str)

            f.write(np.array([total_size_in_bits], dtype=np.uint32).tobytes())
            f.flush()

            padding_length = 0
            if len(all_huffman_codes_str) % 8 != 0:
                padding_length = 8 - (len(all_huffman_codes_str) % 8)
            all_huffman_codes_str = all_huffman_codes_str + '0' * padding_length

            bitstream = bitstring.BitStream(bin=all_huffman_codes_str)
            f.write(bitstream.bytes)  # Write the Huffman code as bytes
            f.flush()

            # To store the RLE Result
            # Store the size of the rle_result
            f.write(np.array([len(rle_result)], dtype=np.uint32).tobytes())  
            f.flush()            

            all_huffman_codes = []
            all_huffman_codes_str = ''

            for count, value in rle_result:
                huffman_code = huffman_codes[value]  # Find Huffman code for this value
                size_in_bits = len(huffman_code)  # The size in bits of the Huffman code

                f.write(np.array([count], dtype=np.uint8).tobytes())
                f.write(np.array([size_in_bits], dtype=np.uint8).tobytes())

                f.flush()

                all_huffman_codes.append(huffman_code)

            all_huffman_codes_str = ''.join(all_huffman_codes)
            total_size_in_bits = len(all_huffman_codes_str)

            # Calculate padding to ensure byte alignment
            if total_size_in_bits % 8 != 0:
                padding_length = 8 - (total_size_in_bits % 8)
                all_huffman_codes_str += '0' * padding_length  # Add padding to the bitstream
            else:
                padding_length = 0

            # Write the total size in bits to the file (before padding)
            f.write(np.array([total_size_in_bits], dtype=np.uint32).tobytes())
            f.flush()

            # Convert the padded bitstream into a byte array
            bitstream = bitstring.BitStream(bin=all_huffman_codes_str)
            f.write(bitstream.bytes)  # Write the Huffman codes as bytes
            f.flush()



    # Finds the original value given the huffman code and the huffman table
    def _decode_huffman_code(self, code_bits, huffman_codes):
        # Reverse the Huffman coding to find the value for the given code bits
        for value, huffman_code in huffman_codes.items():
            if code_bits == huffman_code:
                return value
        raise ValueError(f"Unknown Huffman code: {code_bits}")

    # takes as input the original image and a quality factor Q
    def compress(self, image, filename):
        image = image.astype('float32') # cast to float
        image = image - 128.0 # rescale the values
        chunks = self._chunkify(image) # break into 8x8 chunks 
        
        flattened_matrices = []
        for chunk in chunks:
            quantized_matrix = self._quantize(self._dct2(chunk))
            flattened_matrix = self._zigzag(quantized_matrix)
            flattened_matrices.append(flattened_matrix)

        self._encode(flattened_matrices, filename)

    def decompress(self, filename):
        with open(filename, 'rb') as f:
            # Read the Quality Factor (Q)
            Q = np.fromfile(f, dtype=np.uint8, count=1)[0]
            
            # Read the Huffman table (number of entries)
            num_huffman_codes = np.fromfile(f, dtype=np.uint16, count=1)[0]

            huffman_codes = {}
            
            table_entries = []
            # Read each Huffman code (value, size)
            for _ in range(num_huffman_codes):
                value = np.fromfile(f, dtype=np.int16, count=1)[0]
                size_in_bits = np.fromfile(f, dtype=np.uint8, count=1)[0]
                table_entries.append((value,size_in_bits))
            
            length_huffman_codes = np.fromfile(f, dtype=np.uint32, count=1)[0]
            huffman_codes_bitstream = f.read((length_huffman_codes + 7) // 8)  # Read the total size in bytes
            huffman_codes_bitstream = bitstring.BitStream(bytes=huffman_codes_bitstream).bin

            current_pos = 0
            for value, size_in_bits in table_entries:
                huffman_codes[value] = huffman_codes_bitstream[current_pos:current_pos+size_in_bits]
                current_pos+=size_in_bits
            

            # get the number of rle values
            num_rle_values = np.fromfile(f, dtype=np.uint32, count=1)[0]

            # 3. Decode the RLE-encoded values and sizes
            rle_result = []
            table_entries = []

            for _ in range(num_rle_values):
                count = np.fromfile(f, dtype=np.uint8, count=1)[0]
                size_in_bits = np.fromfile(f, dtype=np.uint8, count=1)[0]
                table_entries.append((count, size_in_bits))
            
            # print(table_entries)
            
            length_huffman_codes = np.fromfile(f, dtype=np.uint32, count=1)[0]
            huffman_codes_bitstream = f.read((length_huffman_codes + 7) // 8)  # Read the total size in bytes
            huffman_codes_bitstream = bitstring.BitStream(bytes=huffman_codes_bitstream).bin

            current_pos = 0
            for count, size_in_bits in table_entries:
                value = self._decode_huffman_code(huffman_codes_bitstream[current_pos:current_pos+size_in_bits], huffman_codes)
                rle_result.append((count, value))
                current_pos+=size_in_bits


            # Reconstruct the original flattened matrices from RLE
            flattened_matrices = []
            
            current_patch = []
            for count, value in rle_result:
                if count == 0 and value == 0:
                    current_patch = current_patch + [0]*(64 - len(current_patch))
                    flattened_matrices.append(np.array(current_patch))
                    current_patch = []
                else:
                    current_patch = current_patch + [0]*count + [value]


            # convert it back to the original patch dimensions
            for i in range(len(flattened_matrices)):
                
                flattened_matrices[i] = self._reverse_zigzag(flattened_matrices[i], 8, 8)

                # now de-quantize the matrix and rescale
                flattened_matrices[i] = (flattened_matrices[i] * self.scaled_quantization_matrix) 
                flattened_matrices[i] = self._idct2(flattened_matrices[i]) + 128
            
            # combine the patches together
            restored_image = self._dechunkify(flattened_matrices, (8*int(np.sqrt(len(flattened_matrices))), 8*int(np.sqrt(len(flattened_matrices)))))
            return restored_image


In [ ]:
image = cv2.imread('data/image1.jpg', cv2.IMREAD_GRAYSCALE)
jpeg = JPEG(Q=100)
jpeg.compress(image, "test")

# Example usage:
filename = "test"
restored_image = jpeg.decompress(filename)

In [ ]:
plt.imshow(restored_image, cmap='gray')  # Use grayscale colormap
plt.axis("off")
plt.show()

# Analysis

### Helpful Functions for Analysis

In [ ]:
def calculate_rmse(original_image, decompressed_image):
    # Flatten the images to 1D arrays for easier comparison
    original_image = original_image.astype(np.float64)
    decompressed_image = decompressed_image.astype(np.float64)

    # Compute the squared differences
    squared_diff = (original_image - decompressed_image) ** 2

    # Calculate the mean squared error (MSE)
    mse = np.mean(squared_diff)

    # RMSE is the square root of MSE
    rmse = np.sqrt(mse)
    
    return rmse

def calculate_bpp(compressed_image_path, image_shape):
    # Get the compressed file size in bits (1 byte = 8 bits)
    file_size_in_bits = os.path.getsize(compressed_image_path) * 8

    # Calculate the total number of pixels in the image
    num_pixels = image_shape[0] * image_shape[1]

    # Compute BPP
    bpp = file_size_in_bits / num_pixels

    return bpp

def psnr(original, compressed):
    original_image = original_image.astype(np.float64)
    decompressed_image = decompressed_image.astype(np.float64)
    mse = np.mean((original - compressed) ** 2)
    if mse == 0:
        return 100  # If MSE is zero, PSNR is infinite (no difference).
    max_pixel = 255.0
    psnr_value = 10 * np.log10((max_pixel ** 2) / mse)
    return psnr_value

### RMSE vs BPP Plots For a Random Set of 20 Images

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Assuming calculate_rmse and calculate_bpp functions are defined elsewhere

# Parameters
quality_factors = range(10, 101, 5)  # Quality factors from 10 to 100 in steps of 10
images_directory = "data/"  # Directory with at least 20 images
output_directory = "compressed_data/"  # Directory to save compressed images
graphs_directory = "graphs/"
compressed_images_directory = "compressed_images/"

# Process images
image_files = [os.path.join(images_directory, f) for f in os.listdir(images_directory)]
image_files = image_files  # Use at least 25 images

# Initialize lists for the all-in-one graph
all_rmse_values = []
all_bpp_values = []
image_labels = []

# Loop through each image and process
for image_path in image_files:
    rmse_values = []
    bpp_values = []
    for q in quality_factors:
        
        jpeg = JPEG(Q=q)  
        original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) 

        # Compress and decompress
        compressed_path = os.path.join(output_directory, f"compressed_q{q}_{os.path.splitext(os.path.basename(image_path))[0]}")
        jpeg.compress(original_image, compressed_path)  
        decompressed_image = jpeg.decompress(compressed_path)  

        decompressed_image_path = os.path.join(compressed_images_directory, f"compressed_q{q}_{os.path.splitext(os.path.basename(image_path))[0]}.jpg")

        # Save the decompressed image as a .jpg file
        cv2.imwrite(decompressed_image_path, decompressed_image)

        if(original_image.shape != decompressed_image.shape):
            print("Error in image_path ", image_path)
            print("Quality Factor ", q)
            continue
        # Calculate RMSE and BPP
        rmse = calculate_rmse(original_image, decompressed_image)  
        bpp = calculate_bpp(compressed_path, original_image.shape)

        rmse_values.append(rmse)
        bpp_values.append(bpp)

    # Plot individual image graph
    plt.figure(figsize=(8, 6))
    plt.plot(bpp_values, rmse_values, marker='o', linestyle='-', label=os.path.splitext(os.path.basename(image_path))[0]) 
    plt.xlabel('Bits Per Pixel (BPP)')
    plt.ylabel('Root Mean Squared Error (RMSE)')
    plt.title(f'RMSE vs BPP for Image {os.path.splitext(os.path.basename(image_path))[0]}')
    plt.grid(True)
    plt.tight_layout()

    # Save the individual plot
    plot_filename = os.path.join(graphs_directory, f"rmse_vs_bpp_{os.path.splitext(os.path.basename(image_path))[0]}.png")
    plt.savefig(plot_filename)
    plt.close()  # Close the figure to avoid overlapping plots

    # Add to the all-in-one graph data
    all_rmse_values.extend(rmse_values)
    all_bpp_values.extend(bpp_values)
    image_labels.extend([os.path.splitext(os.path.basename(image_path))[0]] * len(rmse_values))




In [ ]:
# Generate All-in-One Graph
plt.figure(figsize=(10, 8))
for i, image_path in enumerate(image_files):
    # Plot all images on the same graph with different labels
    start_idx = i * len(quality_factors)
    end_idx = (i + 1) * len(quality_factors)
    plt.plot(all_bpp_values[start_idx:end_idx], all_rmse_values[start_idx:end_idx], marker='o', linestyle='-', label=image_labels[start_idx])

plt.xlabel('Bits Per Pixel (BPP)')
plt.ylabel('Root Mean Squared Error (RMSE)')
plt.title('RMSE vs BPP for All Images')
plt.grid(True)
plt.legend(title="Images", bbox_to_anchor=(1.05, 1), loc='upper left')  # Display legends outside the plot
plt.tight_layout()

# Save the all-in-one plot
all_in_one_plot_filename = os.path.join(graphs_directory, "rmse_vs_bpp_all_images.png")
plt.savefig(all_in_one_plot_filename)
plt.close()

### Compression Rate vs Quality Factor for a Random Set of 20 Images 

In [ ]:
import os
import matplotlib.pyplot as plt

# Define paths
compressed_files_directory = 'compressed_data/'  # Folder containing compressed images
original_image_directory = 'data/'  # Folder containing original images
output_plots_directory = 'graphs/'  # Folder to save generated plots

# Ensure the output directory exists
os.makedirs(output_plots_directory, exist_ok=True)

# Define quality factors
quality_factors = range(10, 101, 5)  # Quality factors from 10 to 100 in steps of 5

# Function to calculate the size of an image in bits
def get_image_size_in_bits(image_path):
    # Get image size in bytes
    size_in_bytes = os.path.getsize(image_path)
    # Convert bytes to bits
    return size_in_bytes * 8

# Get a list of original image files
image_files = [f for f in os.listdir(original_image_directory)]

# Loop through each image and calculate compression rate
for image_file in image_files:
    original_image_path = os.path.join(original_image_directory, image_file)
    original_image = plt.imread(original_image_path)
    original_image_size_bits = original_image.shape[0] * original_image.shape[1] * 8  # Height * Width * Bits per pixel

    # Store compression rates for the current image
    compression_rates_for_image = []

    # Loop through each quality factor
    for q in quality_factors:
        # Find the corresponding compressed file
        compressed_file_name = f"compressed_q{q}_{os.path.splitext(image_file)[0]}"
        compressed_image_path = os.path.join(compressed_files_directory, compressed_file_name)

        if os.path.exists(compressed_image_path):
            compressed_image_size_bits = get_image_size_in_bits(compressed_image_path)
            compression_rate = original_image_size_bits / compressed_image_size_bits
            compression_rates_for_image.append(compression_rate)
        else:
            print(f"Compressed file for quality factor {q} not found for {image_file}.")

    # Plot Compression Rate vs Quality Factor for the current image
    plt.figure(figsize=(8, 6))  # Optional: Adjust the size of the plot
    plt.plot(quality_factors, compression_rates_for_image, marker='o', label=image_file, color='b')
    
    # Add labels and title
    plt.xlabel('Quality Factor')
    plt.ylabel('Compression Rate')
    plt.title(f'Compression Rate vs Quality Factor\n{image_file}')
    
    # Save the plot as an image file
    plot_file_path = os.path.join(output_plots_directory, f'compression_rate_vs_q_{os.path.splitext(image_file)[0]}.png')
    plt.legend()
    plt.grid(True)
    plt.savefig(plot_file_path)  # Save the plot to the output directory
    
    # Close the plot to avoid overlapping plots
    plt.close()

    print(f"Saved plot for {image_file} at {plot_file_path}")


In [ ]:
import os
import matplotlib.pyplot as plt

# Define paths
compressed_files_directory = 'compressed_data/'  # Folder containing compressed images
original_image_directory = 'data/'  # Folder containing original images
output_plots_directory = 'graphs/'  # Folder to save generated plots

# Ensure the output directory exists
os.makedirs(output_plots_directory, exist_ok=True)

# Define quality factors
quality_factors = range(10, 101, 5)  # Quality factors from 10 to 100 in steps of 5

# Function to calculate the size of an image in bits
def get_image_size_in_bits(image_path):
    # Get image size in bytes
    size_in_bytes = os.path.getsize(image_path)
    # Convert bytes to bits
    return size_in_bytes * 8

# Get a list of original image files
image_files = [os.path.join(images_directory, f) for f in os.listdir(images_directory)]

# Initialize a plot for all images
plt.figure(figsize=(10, 8))  # Optional: Adjust the size of the plot

# Loop through each image and calculate compression rate
for image_file in image_files:
    original_image_path = os.path.join(original_image_directory, image_file)
    original_image = plt.imread(original_image_path)
    original_image_size_bits = original_image.shape[0] * original_image.shape[1] * 8  # Height * Width * Bits per pixel

    # Store compression rates for the current image
    compression_rates_for_image = []

    # Loop through each quality factor
    for q in quality_factors:
        # Find the corresponding compressed file
        compressed_file_name = f"compressed_q{q}_{os.path.splitext(image_file)[0]}"
        compressed_image_path = os.path.join(compressed_files_directory, compressed_file_name)

        if os.path.exists(compressed_image_path):
            compressed_image_size_bits = get_image_size_in_bits(compressed_image_path)
            compression_rate = original_image_size_bits / compressed_image_size_bits
            compression_rates_for_image.append(compression_rate)
        else:
            print(f"Compressed file for quality factor {q} not found for {image_file}.")

    # Plot Compression Rate vs Quality Factor for the current image, overlayed on the same plot
    plt.plot(quality_factors, compression_rates_for_image, marker='o', label=image_file)

# Add labels and title to the overall plot
plt.xlabel('Quality Factor')
plt.ylabel('Compression Rate')
plt.title('Compression Rate vs Quality Factor for All Images')

# Show a legend and grid
plt.legend()
plt.grid(True)

# Save the plot as an image file
plot_file_path = os.path.join(output_plots_directory, 'compression_rate_vs_q_all_images.png')
plt.savefig(plot_file_path)  # Save the plot to the output directory

# Close the plot to avoid overlapping plots
plt.close()

print(f"Saved overlayed plot for all images at {plot_file_path}")


### Comparision with JPEG

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Parameters
quality_factors = range(10, 101, 5)  # Quality factors from 10 to 100 in steps of 5
images_directory = "data/"  # Directory with at least 20 images
output_directory = "compressed_images_bin/"  # Directory to save compressed images
graphs_directory = "graphs_comparison/"
compressed_images_directory = "compressed_images/"

# Process images
image_files = [os.path.join(images_directory, f) for f in os.listdir(images_directory)]

# Loop through each image and process
for image_path in image_files:
    rmse_values_cv2 = []
    bpp_values_cv2 = []
    rmse_values_custom = []
    bpp_values_custom = []
    compression_rate_cv2 = []
    compression_rate_custom = []
    
    for q in quality_factors:
        original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # OpenCV Compression (cv2)
        encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), q]
        _, compressed_image_cv2 = cv2.imencode('.jpg', original_image, encode_param)
        
        # Save the OpenCV compressed image
        compressed_path_cv2 = os.path.join(output_directory, f"cv2_compressed_q{q}_{os.path.splitext(os.path.basename(image_path))[0]}.jpg")
        with open(compressed_path_cv2, 'wb') as f:
            f.write(compressed_image_cv2.tobytes())
        
        # Decompress using OpenCV (cv2)
        decompressed_image_cv2 = cv2.imdecode(compressed_image_cv2, cv2.IMREAD_GRAYSCALE)

        # Calculate RMSE and BPP for OpenCV method
        rmse_cv2 = calculate_rmse(original_image, decompressed_image_cv2)
        bpp_cv2 = calculate_bpp(compressed_path_cv2, original_image.shape)

        # Calculate Compression Rate for OpenCV
        original_size_cv2 = os.path.getsize(image_path)
        compressed_size_cv2 = os.path.getsize(compressed_path_cv2)
        compression_rate_cv2.append(original_size_cv2 / compressed_size_cv2)

        rmse_values_cv2.append(rmse_cv2)
        bpp_values_cv2.append(bpp_cv2)

        # Custom JPEG Compression (assuming custom JPEG class is defined)
        jpeg = JPEG(Q=q)  # Assuming your custom JPEG implementation is called 'JPEG'
        
        compressed_path_custom = os.path.join(output_directory, f"custom_compressed_q{q}_{os.path.splitext(os.path.basename(image_path))[0]}")
        jpeg.compress(original_image, compressed_path_custom)  
        decompressed_image_custom = jpeg.decompress(compressed_path_custom)  # Decompress using custom method

        decompressed_image_path_custom = os.path.join(compressed_images_directory, f"custom_compressed_q{q}_{os.path.splitext(os.path.basename(image_path))[0]}.jpg")

        # Save the decompressed custom image as a .jpg file
        cv2.imwrite(decompressed_image_path_custom, decompressed_image_custom)

        # Check for shape mismatch
        if original_image.shape != decompressed_image_custom.shape:
            print(f"Error in image_path: {image_path}, Quality Factor: {q}")
            continue

        # Calculate RMSE and BPP for custom method
        rmse_custom = calculate_rmse(original_image, decompressed_image_custom)
        bpp_custom = calculate_bpp(compressed_path_custom, original_image.shape)

        # Calculate Compression Rate for custom method
        original_size_custom = os.path.getsize(image_path)
        compressed_size_custom = os.path.getsize(compressed_path_custom)
        compression_rate_custom.append(original_size_custom / compressed_size_custom)
    
        rmse_values_custom.append(rmse_custom)
        bpp_values_custom.append(bpp_custom)

    # Plot the comparison of RMSE vs BPP for both methods (OpenCV vs Custom JPEG)
    plt.figure(figsize=(8, 6))
    plt.plot(bpp_values_cv2, rmse_values_cv2, marker='o', linestyle='-', label='OpenCV JPEG', color='blue')
    plt.plot(bpp_values_custom, rmse_values_custom, marker='x', linestyle='-', label='Custom JPEG', color='red')
    plt.xlabel('Bits Per Pixel (BPP)')
    plt.ylabel('Root Mean Squared Error (RMSE)')
    plt.title(f'RMSE vs BPP for Image {os.path.splitext(os.path.basename(image_path))[0]}')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    # Save the comparison plot
    plot_filename = os.path.join(graphs_directory, f"rmse_vs_bpp_comparison_{os.path.splitext(os.path.basename(image_path))[0]}.png")
    plt.savefig(plot_filename)
    plt.close()  # Close the figure to avoid overlapping plots

    # Plot Compression Rate vs Quality Factor for both methods (OpenCV vs Custom JPEG)
    plt.figure(figsize=(8, 6))
    plt.plot(quality_factors, compression_rate_cv2, marker='o', linestyle='-', label='OpenCV JPEG', color='blue')
    plt.plot(quality_factors, compression_rate_custom, marker='x', linestyle='-', label='Custom JPEG', color='red')
    plt.xlabel('Quality Factor')
    plt.ylabel('Compression Rate')
    plt.title(f'Compression Rate vs Quality Factor for Image {os.path.splitext(os.path.basename(image_path))[0]}')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    # Save the compression rate plot
    rate_plot_filename = os.path.join(graphs_directory, f"compression_rate_vs_q_{os.path.splitext(os.path.basename(image_path))[0]}.png")
    plt.savefig(rate_plot_filename)
    plt.close()  # Close the figure to avoid overlapping plots


### Variation over Quantization Matrices

In [ ]:
class JPEG_Exp:
    def __init__(self, Q = 50, quantization_matrix = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                                                [12, 12, 14, 19, 26, 58, 60, 55],
                                                [14, 13, 16, 24, 40, 57, 69, 56],
                                                [14, 17, 22, 29, 51, 87, 80, 62],
                                                [18, 22, 37, 56, 68, 109, 103, 77],
                                                [24, 35, 55, 64, 81, 104, 113, 92],
                                                [49, 64, 78, 87, 103, 121, 120, 101],
                                                [72, 92, 95, 98, 112, 100, 103, 99],])):
        self.Q = Q
        self.quantization_matrix = quantization_matrix
        self.scaled_quantization_matrix = np.round(self.quantization_matrix * (50.0/Q))
    
    # define the dct and idct functions for the blocks
    def _dct2(self, block):
        dct_val = dct(dct(block.T, norm='ortho').T, norm='ortho')
        return dct_val

    def _idct2(self, block):
        return idct(idct(block.T, norm='ortho').T, norm='ortho')
    
    # Runs quantization while compression
    def _quantize(self, block):
        return np.round(block / self.scaled_quantization_matrix)

    # Runs dequantization while decompression
    def _dequantize(self, block):
        return block * self.scaled_quantization_matrix
    
    # breaks the image into 8x8 patches
    def _chunkify(self, image):
        chunks = []
        h,w = np.shape(image)
        for i in range(0, h, 8):
            for j in range(0, w, 8):
                chunk = image[i:i+8, j:j+8]
                chunks.append(chunk)

        return chunks

    # recombines the patches into the original image size
    def _dechunkify(self, chunks, original_shape):
        h, w = original_shape
        image = np.zeros((h, w), dtype=chunks[0].dtype)  # Create an empty array with the original shape
        chunk_idx = 0  # Index to keep track of the current chunk

        for i in range(0, h, 8):
            for j in range(0, w, 8):
                # Place the current chunk into the corresponding position in the image
                image[i:i+8, j:j+8] = chunks[chunk_idx]
                chunk_idx += 1

        return image

    # flattens the matrix into its zig-zag traversal
    def _zigzag(self, matrix):
        rows, cols = matrix.shape
        result = []

        for s in range(rows + cols - 1):
            if s % 2 == 0:  # Even diagonals (move up-right)
                x = min(s, rows - 1)
                y = s - x
                while x >= 0 and y < cols:
                    result.append(matrix[x, y])
                    x -= 1
                    y += 1
            else:  # Odd diagonals (move down-left)
                y = min(s, cols - 1)
                x = s - y
                while y >= 0 and x < rows:
                    result.append(matrix[x, y])
                    x += 1
                    y -= 1

        return np.array(result)

    # converts a flattened matrix into the original form by placing the elements in a zig zag manner
    def _reverse_zigzag(self, flattened, rows, cols):
        # Initialize an empty 2D matrix
        matrix = np.zeros((rows, cols), dtype=flattened.dtype)

        # Fill the matrix using the reverse zig-zag order
        index = 0
        for s in range(rows + cols - 1):
            if s % 2 == 0:  # Even diagonals (move up-right)
                x = min(s, rows - 1)
                y = s - x
                while x >= 0 and y < cols:
                    matrix[x, y] = flattened[index]
                    index += 1
                    x -= 1
                    y += 1
            else:  # Odd diagonals (move down-left)
                y = min(s, cols - 1)
                x = s - y
                while y >= 0 and x < rows:
                    matrix[x, y] = flattened[index]
                    index += 1
                    x += 1
                    y -= 1

        return matrix
    
    # Helper functions for huffman encoding and RLE
    def _build_huffman_tree(self, frequencies):
        # Create a priority queue (min-heap) to build the Huffman tree
        heap = [[weight, [symbol, ""]] for symbol, weight in frequencies.items()]
        heapq.heapify(heap)
        
        while len(heap) > 1:
            low = heapq.heappop(heap)
            high = heapq.heappop(heap)
            for pair in low[1:]:
                pair[1] = '0' + pair[1]
            for pair in high[1:]:
                pair[1] = '1' + pair[1]
            heapq.heappush(heap, [low[0] + high[0]] + low[1:] + high[1:])
        
        # Generate the Huffman codes
        huffman_codes = {}
        for symbol, code in heap[0][1:]:
            huffman_codes[symbol] = code
        return huffman_codes

    def _run_length_encoding(self, matrix):
        rle = []
        zero_count = 0
        
        for value in matrix:
            if value == 0:
                zero_count += 1  # Increment zero counter
                if zero_count == 16:
                    rle.append((15, 0))
                    zero_count = 0
            else:
                rle.append((zero_count, value))  # Store the number of zeros before the value
                zero_count = 0  # Reset zero counter after encountering a non-zero value
        
        return rle

    def _huffman_encoding(self, values):
        frequencies = defaultdict(int)
        for val in values:
            frequencies[val] += 1
        
        huffman_codes = self._build_huffman_tree(frequencies)
        return huffman_codes

    # Function to encode the image post scaling dct and quantization into rle and huffman and store into a bin file
    def _encode(self, flattened_matrices, filename):
        with open(filename, 'wb') as f:
            rle_result = []

            for i in range(0,len(flattened_matrices)):
                rle = self._run_length_encoding(flattened_matrices[i])
                rle_result.extend(rle)
                rle_result.append((0,0))

            values = [value for count, value in rle_result]  # Only values for Huffman encoding
            huffman_codes = self._huffman_encoding(values)    

            # Store the Quality Factor in the file 
            f.write(np.array([self.Q], dtype=np.uint8).tobytes()) 
            f.flush()
        

            # Store the Huffman table in the file (value -> huffman code mapping)
            f.write(np.array([len(huffman_codes)], dtype=np.uint16).tobytes())  # Write the number of unique values in Huffman table
            f.flush()            
 

            # Collect all Huffman codes
            all_huffman_codes = []

            for value, code in huffman_codes.items():
                f.write(np.array([value], dtype=np.int16).tobytes())  
                f.flush()
                size_in_bits = len(code)
                f.write(np.array([size_in_bits], dtype=np.uint8).tobytes())  # Write the size of the Huffman code
                f.flush()
                all_huffman_codes.append(code)

            all_huffman_codes_str = ''.join(all_huffman_codes)
            total_size_in_bits = len(all_huffman_codes_str)

            f.write(np.array([total_size_in_bits], dtype=np.uint32).tobytes())
            f.flush()

            padding_length = 0
            if len(all_huffman_codes_str) % 8 != 0:
                padding_length = 8 - (len(all_huffman_codes_str) % 8)
            all_huffman_codes_str = all_huffman_codes_str + '0' * padding_length

            bitstream = bitstring.BitStream(bin=all_huffman_codes_str)
            f.write(bitstream.bytes)  # Write the Huffman code as bytes
            f.flush()

            # To store the RLE Result
            # Store the size of the rle_result
            f.write(np.array([len(rle_result)], dtype=np.uint32).tobytes())  
            f.flush()            

            all_huffman_codes = []
            all_huffman_codes_str = ''

            for count, value in rle_result:
                huffman_code = huffman_codes[value]  # Find Huffman code for this value
                size_in_bits = len(huffman_code)  # The size in bits of the Huffman code

                f.write(np.array([count], dtype=np.uint8).tobytes())
                f.write(np.array([size_in_bits], dtype=np.uint8).tobytes())

                f.flush()

                all_huffman_codes.append(huffman_code)

            all_huffman_codes_str = ''.join(all_huffman_codes)
            total_size_in_bits = len(all_huffman_codes_str)

            # Calculate padding to ensure byte alignment
            if total_size_in_bits % 8 != 0:
                padding_length = 8 - (total_size_in_bits % 8)
                all_huffman_codes_str += '0' * padding_length  # Add padding to the bitstream
            else:
                padding_length = 0

            # Write the total size in bits to the file (before padding)
            f.write(np.array([total_size_in_bits], dtype=np.uint32).tobytes())
            f.flush()

            # Convert the padded bitstream into a byte array
            bitstream = bitstring.BitStream(bin=all_huffman_codes_str)
            f.write(bitstream.bytes)  # Write the Huffman codes as bytes
            f.flush()



    # Finds the original value given the huffman code and the huffman table
    def _decode_huffman_code(self, code_bits, huffman_codes):
        # Reverse the Huffman coding to find the value for the given code bits
        for value, huffman_code in huffman_codes.items():
            if code_bits == huffman_code:
                return value
        raise ValueError(f"Unknown Huffman code: {code_bits}")

    # takes as input the original image and a quality factor Q
    def compress(self, image, filename):
        image = image.astype('float32') # cast to float
        image = image - 128.0 # rescale the values
        chunks = self._chunkify(image) # break into 8x8 chunks 
        
        flattened_matrices = []
        for chunk in chunks:
            quantized_matrix = self._quantize(self._dct2(chunk))
            flattened_matrix = self._zigzag(quantized_matrix)
            flattened_matrices.append(flattened_matrix)

        self._encode(flattened_matrices, filename)

    def decompress(self, filename):
        with open(filename, 'rb') as f:
            # Read the Quality Factor (Q)
            Q = np.fromfile(f, dtype=np.uint8, count=1)[0]
            
            # Read the Huffman table (number of entries)
            num_huffman_codes = np.fromfile(f, dtype=np.uint16, count=1)[0]

            huffman_codes = {}
            
            table_entries = []
            # Read each Huffman code (value, size)
            for _ in range(num_huffman_codes):
                value = np.fromfile(f, dtype=np.int16, count=1)[0]
                size_in_bits = np.fromfile(f, dtype=np.uint8, count=1)[0]
                table_entries.append((value,size_in_bits))
            
            length_huffman_codes = np.fromfile(f, dtype=np.uint32, count=1)[0]
            huffman_codes_bitstream = f.read((length_huffman_codes + 7) // 8)  # Read the total size in bytes
            huffman_codes_bitstream = bitstring.BitStream(bytes=huffman_codes_bitstream).bin

            current_pos = 0
            for value, size_in_bits in table_entries:
                huffman_codes[value] = huffman_codes_bitstream[current_pos:current_pos+size_in_bits]
                current_pos+=size_in_bits
            

            # get the number of rle values
            num_rle_values = np.fromfile(f, dtype=np.uint32, count=1)[0]

            # 3. Decode the RLE-encoded values and sizes
            rle_result = []
            table_entries = []

            for _ in range(num_rle_values):
                count = np.fromfile(f, dtype=np.uint8, count=1)[0]
                size_in_bits = np.fromfile(f, dtype=np.uint8, count=1)[0]
                table_entries.append((count, size_in_bits))
            
            # print(table_entries)
            
            length_huffman_codes = np.fromfile(f, dtype=np.uint32, count=1)[0]
            huffman_codes_bitstream = f.read((length_huffman_codes + 7) // 8)  # Read the total size in bytes
            huffman_codes_bitstream = bitstring.BitStream(bytes=huffman_codes_bitstream).bin

            current_pos = 0
            for count, size_in_bits in table_entries:
                value = self._decode_huffman_code(huffman_codes_bitstream[current_pos:current_pos+size_in_bits], huffman_codes)
                rle_result.append((count, value))
                current_pos+=size_in_bits


            # Reconstruct the original flattened matrices from RLE
            flattened_matrices = []
            
            current_patch = []
            for count, value in rle_result:
                if count == 0 and value == 0:
                    current_patch = current_patch + [0]*(64 - len(current_patch))
                    flattened_matrices.append(np.array(current_patch))
                    current_patch = []
                else:
                    current_patch = current_patch + [0]*count + [value]


            # convert it back to the original patch dimensions
            for i in range(len(flattened_matrices)):
                
                flattened_matrices[i] = self._reverse_zigzag(flattened_matrices[i], 8, 8)

                # now de-quantize the matrix and rescale
                flattened_matrices[i] = (flattened_matrices[i] * self.scaled_quantization_matrix) 
                flattened_matrices[i] = self._idct2(flattened_matrices[i]) + 128
            
            # combine the patches together
            restored_image = self._dechunkify(flattened_matrices, (8*int(np.sqrt(len(flattened_matrices))), 8*int(np.sqrt(len(flattened_matrices)))))
            return restored_image


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from PIL import Image
import time
import shutil

# Assuming JPEG_Exp is the class you have implemented
# Define quantization matrices
JPEG_STANDARD = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                          [12, 12, 14, 19, 26, 58, 60, 55],
                          [14, 13, 16, 24, 40, 57, 69, 56],
                          [14, 17, 22, 29, 51, 87, 80, 62],
                          [18, 22, 37, 56, 68, 109, 103, 77],
                          [24, 35, 55, 64, 81, 104, 113, 92],
                          [49, 64, 78, 87, 103, 121, 120, 101],
                          [72, 92, 95, 98, 112, 100, 103, 99]])

HIGHLY_DETAILED = np.array([[4, 3, 3, 4, 6, 10, 13, 15],
                            [3, 3, 4, 5, 7, 16, 16, 14],
                            [4, 4, 5, 7, 10, 15, 18, 15],
                            [4, 5, 6, 8, 14, 24, 23, 19],
                            [6, 7, 11, 17, 20, 32, 30, 23],
                            [10, 16, 25, 29, 37, 48, 53, 41],
                            [13, 16, 20, 23, 28, 56, 55, 46],
                            [15, 14, 15, 19, 23, 41, 43, 41]])

LOW_DETAIL = np.array([[50, 40, 35, 50, 60, 80, 90, 100],
                       [40, 40, 50, 60, 70, 100, 110, 90],
                       [45, 50, 60, 75, 85, 100, 120, 100],
                       [50, 60, 70, 90, 110, 150, 140, 120],
                       [60, 70, 90, 110, 130, 180, 170, 140],
                       [80, 100, 130, 140, 160, 200, 210, 160],
                       [90, 110, 130, 150, 170, 220, 240, 190],
                       [100, 120, 140, 160, 180, 220, 240, 210]])

JPEG2000_LIKE = np.array([[9, 5, 4, 6, 8, 12, 16, 18],
                          [5, 5, 6, 8, 10, 18, 19, 16],
                          [6, 6, 8, 10, 14, 21, 24, 20],
                          [6, 8, 10, 13, 17, 27, 26, 21],
                          [8, 10, 14, 17, 21, 31, 29, 23],
                          [12, 18, 21, 24, 30, 39, 42, 32],
                          [16, 19, 24, 26, 33, 43, 44, 36],
                          [18, 16, 20, 21, 26, 36, 39, 35]])

UNIFORM = np.ones((8, 8)) * 20

# Set up experiment parameters
quality_factors = [10, 20, 30, 50, 75]
quantization_matrices = {
    "JPEG Standard": JPEG_STANDARD,
    "Highly Detailed": HIGHLY_DETAILED,
    "Low Detail": LOW_DETAIL,
    "JPEG2000-like": JPEG2000_LIKE,
    "Uniform": UNIFORM
}

# Create folder to store results
output_folder = "compression_results"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to calculate compression rate (CR)
def calculate_compression_rate(original_size, compressed_size):
    return original_size / compressed_size

# Function to calculate RMSE
def calculate_rmse(original_image, decompressed_image):
    return np.sqrt(mean_squared_error(original_image.flatten(), decompressed_image.flatten()))

# Loop through all BMP files in the folder
input_folder = "data"
bmp_files = [f for f in os.listdir(input_folder) if f.endswith(".bmp")]

# Initialize lists to store metrics for plotting
cr_results = {matrix_name: [] for matrix_name in quantization_matrices.keys()}
rmse_results = {matrix_name: [] for matrix_name in quantization_matrices.keys()}
bpp_results = {matrix_name: [] for matrix_name in quantization_matrices.keys()}

# Run the experiment for each image
for file in bmp_files:
    # Load the image
    image_path = os.path.join(input_folder, file)
    img = np.array(Image.open(image_path).convert('L'))  # Convert to grayscale

    for matrix_name, matrix in quantization_matrices.items():
        for Q in quality_factors:
            # Instantiate the JPEG_Exp class with the specific quantization matrix and quality factor
            jpeg_exp = JPEG_Exp(Q=Q, quantization_matrix=matrix)
            
            # Compress the image
            compressed_file = os.path.join(output_folder, f"{file}_compressed_Q{Q}_{matrix_name}.bin")
            jpeg_exp.compress(img, compressed_file)

            # Decompress the image
            decompressed_img = jpeg_exp.decompress(compressed_file)

            # Calculate RMSE
            rmse = calculate_rmse(img, decompressed_img)

            # Calculate compression rate
            original_size = img.nbytes
            compressed_size = os.path.getsize(compressed_file)
            cr = calculate_compression_rate(original_size, compressed_size)

            # Calculate BPP (Bits Per Pixel)
            bpp = compressed_size * 8 / img.size  # in bits per pixel

            # Store results
            cr_results[matrix_name].append(cr)
            rmse_results[matrix_name].append(rmse)
            bpp_results[matrix_name].append(bpp)

            # Save the Q=50 compressed image for each matrix
            if Q == 50:
                q50_image_folder = os.path.join(output_folder, f"Q50_{matrix_name}")
                if not os.path.exists(q50_image_folder):
                    os.makedirs(q50_image_folder)
                q50_image_path = os.path.join(q50_image_folder, f"{file}")
                Image.fromarray(decompressed_img.astype(np.uint8)).save(q50_image_path)


In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np

# Assuming these are your concatenated results
# cr_results = {"matrix_name": [concatenated data]}
# rmse_results = {"matrix_name": [concatenated data]}
# bpp_results = {"matrix_name": [concatenated data]}
# Replace this with the actual dictionaries you have

# Number of images
num_images = len(bmp_files)

# Number of quality factors
num_quality_factors = len(quality_factors)

# Restructure results
image_results = {file: {"cr_results": {}, "rmse_results": {}, "bpp_results": {}} for file in bmp_files}

for matrix_name in quantization_matrices.keys():
    # Split the data for each metric
    cr_data = np.array(cr_results[matrix_name])
    rmse_data = np.array(rmse_results[matrix_name])
    bpp_data = np.array(bpp_results[matrix_name])

    # Reshape the data into [num_images, num_quality_factors]
    cr_split = cr_data.reshape(num_images, num_quality_factors)
    rmse_split = rmse_data.reshape(num_images, num_quality_factors)
    bpp_split = bpp_data.reshape(num_images, num_quality_factors)

    for idx, file in enumerate(bmp_files):
        image_results[file]["cr_results"][matrix_name] = cr_split[idx, :]
        image_results[file]["rmse_results"][matrix_name] = rmse_split[idx, :]
        image_results[file]["bpp_results"][matrix_name] = bpp_split[idx, :]

# Generate and save plots for each image
output_folder = "compression_results"

for file in bmp_files:
    metrics = image_results[file]
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Compression Rate vs Q Factor
    for matrix_name in quantization_matrices.keys():
        axes[0].plot(quality_factors, metrics["cr_results"][matrix_name], label=matrix_name)
    axes[0].set_title(f'Compression Rate vs Q Factor\n{file}')
    axes[0].set_xlabel('Q Factor')
    axes[0].set_ylabel('Compression Rate')
    axes[0].legend()

    # RMSE vs BPP
    for matrix_name in quantization_matrices.keys():
        axes[1].plot(metrics["bpp_results"][matrix_name], metrics["rmse_results"][matrix_name], label=matrix_name)
    axes[1].set_title(f'RMSE vs BPP\n{file}')
    axes[1].set_xlabel('BPP')
    axes[1].set_ylabel('RMSE')
    axes[1].legend()

    # Save the plots
    plot_path = os.path.join(output_folder, f"{os.path.splitext(file)[0]}_plots.png")
    plt.savefig(plot_path)
    plt.close(fig)


In [ ]:
## Collage
import os
from PIL import Image
import matplotlib.pyplot as plt

# Define paths
input_folder = "data"  # Folder containing original images
folders = ["Q50_JPEG Standard", "Q50_Highly Detailed", "Q50_Low Detail", "Q50_JPEG2000-like", "Q50_Uniform"]
output_folder = "collages"
os.makedirs(output_folder, exist_ok=True)

# Get list of original images
original_images = [f for f in os.listdir(input_folder) if f.endswith(".bmp")]

# Create a collage for each image
for original_image in original_images:
    # Load the original image
    original_path = os.path.join(input_folder, original_image)
    original = Image.open(original_path)

    # Load Q=50 images from each folder
    q50_images = []
    for folder in folders:
        q50_path = os.path.join("compression_results", folder, original_image)
        if os.path.exists(q50_path):
            q50_images.append(Image.open(q50_path))
        else:
            print(f"Warning: File {q50_path} not found.")
            q50_images.append(None)

    # Create a 4x2 collage
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f"Collage for {original_image}", fontsize=16)

    # Place Original image in the first position
    axes[0, 0].imshow(original, cmap="gray")
    axes[0, 0].set_title("Original")
    axes[0, 0].axis("off")

    # Place Q=50 images
    for idx, (ax, matrix_name, q50_img) in enumerate(zip(axes.flatten()[1:], folders, q50_images)):
        if q50_img:
            ax.imshow(q50_img, cmap="gray")
            ax.set_title(f"{matrix_name} (Q=50)")
        else:
            ax.set_title(f"{matrix_name} (Missing)")
        ax.axis("off")

    # Remove any unused subplots
    for ax in axes.flatten()[len(q50_images) + 1:]:
        fig.delaxes(ax)

    # Save the collage
    collage_path = os.path.join(output_folder, f"{os.path.splitext(original_image)[0]}_collage.png")
    plt.tight_layout()
    plt.savefig(collage_path)
    plt.close(fig)


### 